In [1]:
import os
import pandas as pd
from datetime import datetime

In [2]:
# Read your combined CSV
CSV_FILE = r'C:\Users\kevin\Desktop\VSCode\Zeru\task_2\output\wallet_txns_combined.csv'
df = pd.read_csv(CSV_FILE)

In [3]:
df.head()

,wallet_id,action,timestamp,value,gas,gasUsed
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,external,1493254201,0.200,333333,21000
1,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,external,1493254443,0.007,27961,23301
2,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,external,1493254495,0.007,27961,23301
3,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,external,1493254558,0.000,61078,35898
4,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,external,1493490509,0.020,120763,100636


In [4]:
df.isnull().sum()

wallet_id    0
action       0
timestamp    0
value        0
gas          0
gasUsed      0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7270 entries, 0 to 7269
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   wallet_id  7270 non-null   object 
 1   action     7270 non-null   object 
 2   timestamp  7270 non-null   int64  
 3   value      7270 non-null   float64
 4   gas        7270 non-null   int64  
 5   gasUsed    7270 non-null   int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 340.9+ KB


In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7270 entries, 0 to 7269
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   wallet_id  7270 non-null   object        
 1   action     7270 non-null   object        
 2   timestamp  7270 non-null   datetime64[ns]
 3   value      7270 non-null   float64       
 4   gas        7270 non-null   int64         
 5   gasUsed    7270 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 340.9+ KB


In [8]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
timestamp,7270,2020-10-24 20:58:15.913342464,2017-04-27 00:50:01,2019-12-26 07:39:16,2020-11-10 05:01:43,2021-06-26 06:05:39.500000,2025-07-22 04:55:11,NaN
value,7270.0,12.484273,0.0,0.0,0.0,0.2,4504.592127,136.803849
gas,7270.0,270729.623109,2300.0,21000.0,88965.0,260010.5,11661875.0,627060.079027
gasUsed,7270.0,176587.833563,0.0,21000.0,49811.0,164190.0,7138144.0,480577.411155


In [9]:
# Feature aggregation per wallet
def compute_wallet_features(df_group):
    feat = {}
    feat['wallet_id'] = df_group.name
    
    feat['days_active'] = (df_group.timestamp.max() - df_group.timestamp.min()).days
    feat['n_txs'] = len(df_group)
    feat['n_external'] = (df_group['action'] == 'external').sum()
    feat['n_internal'] = (df_group['action'] == 'internal').sum()

    feat['total_value_eth'] = df_group['value'].sum()
    feat['avg_gas'] = df_group['gas'].mean()
    feat['total_gas_used'] = df_group['gasUsed'].sum()
    feat['last_tx_days_ago'] = (datetime.utcnow() - df_group.timestamp.max()).days
    
    return feat

In [10]:
wallet_feats = df.groupby('wallet_id').apply(compute_wallet_features)
wallet_feats = pd.DataFrame(wallet_feats.tolist())
wallet_feats.head()

C:\Users\kevin\AppData\Local\Temp\ipykernel_20648\476966291.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  feat['last_tx_days_ago'] = (datetime.utcnow() - df_group.timestamp.max()).days


,wallet_id,days_active,n_txs,n_external,n_internal,total_value_eth,avg_gas,total_gas_used,last_tx_days_ago
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,2513,2246,1946,300,14876.959377,438803.196794,656379467,498
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0,4,4,0,0.024302,91256.000000,266363,1743
2,0x0795732aacc448030ef374374eaae57d2965c16c,0,3,3,0,0.018600,114674.666667,248424,1702
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,1879,28,24,4,270.312981,131456.107143,2433893,309
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,63,4,4,0,0.026005,91256.000000,266363,1673


In [11]:
wallet_feats.describe().T

,count,mean,std,min,25%,50%,75%,max
days_active,103.0,4.595922e+02,7.205122e+02,0.0,0.000000,49.000000,6.630000e+02,2.862000e+03
n_txs,103.0,7.058252e+01,2.459461e+02,2.0,4.000000,6.000000,2.350000e+01,2.246000e+03
n_external,103.0,6.234951e+01,2.147178e+02,2.0,3.000000,5.000000,2.000000e+01,1.946000e+03
n_internal,103.0,8.233010e+00,3.270972e+01,0.0,0.000000,0.000000,2.000000e+00,3.000000e+02
total_value_eth,103.0,8.811715e+02,5.238988e+03,0.0,0.020852,0.033459,1.842860e+00,4.458180e+04
avg_gas,103.0,1.562414e+05,1.427729e+05,49102.4,91256.000000,114674.666667,1.343765e+05,1.037290e+06
total_gas_used,103.0,1.246402e+07,6.644877e+07,156931.0,266363.000000,437785.000000,2.061207e+06,6.563795e+08
last_tx_days_ago,103.0,1.385592e+03,5.481444e+02,3.0,1303.500000,1680.000000,1.716000e+03,2.197000e+03


In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols = ['days_active', 'n_txs', 'total_value_eth', 'total_gas_used', 'last_tx_days_ago']
features_scaled = scaler.fit_transform(wallet_feats[cols])
scaled_df = pd.DataFrame(features_scaled, columns=[f"{c}_norm" for c in cols])
features = pd.concat([wallet_feats, scaled_df], axis=1)

In [13]:
features.head()

,wallet_id,days_active,n_txs,n_external,n_internal,total_value_eth,avg_gas,total_gas_used,last_tx_days_ago,days_active_norm,n_txs_norm,total_value_eth_norm,total_gas_used_norm,last_tx_days_ago_norm
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,2513,2246,1946,300,14876.959377,438803.196794,656379467,498,0.878057,1.000000,3.337003e-01,1.000000,0.225615
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0,4,4,0,0.024302,91256.000000,266363,1743,0.000000,0.000891,5.451070e-07,0.000167,0.793072
2,0x0795732aacc448030ef374374eaae57d2965c16c,0,3,3,0,0.018600,114674.666667,248424,1702,0.000000,0.000446,4.172106e-07,0.000139,0.774385
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,1879,28,24,4,270.312981,131456.107143,2433893,309,0.656534,0.011586,6.063304e-03,0.003470,0.139471
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,63,4,4,0,0.026005,91256.000000,266363,1673,0.022013,0.000891,5.833029e-07,0.000167,0.761167


In [14]:
def compute_score(row):
    score = (
        (1 - row['days_active_norm']) * 0.2 +
        row['n_txs_norm'] * 0.2 +
        row['total_value_eth_norm'] * 0.25 +
        row['total_gas_used_norm'] * 0.15 +
        row['last_tx_days_ago_norm'] * 0.2
    )
    return int(score * 1000)

In [15]:
features['score'] = features.apply(compute_score, axis=1)

In [16]:
output_file = r'C:\Users\kevin\Desktop\VSCode\Zeru\task_2\output'
features[['wallet_id', 'score']].to_csv(os.path.join(output_file, "wallet_risk_scores.csv"), index=False)
features.head()

,wallet_id,days_active,n_txs,n_external,n_internal,total_value_eth,avg_gas,total_gas_used,last_tx_days_ago,days_active_norm,n_txs_norm,total_value_eth_norm,total_gas_used_norm,last_tx_days_ago_norm,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,2513,2246,1946,300,14876.959377,438803.196794,656379467,498,0.878057,1.000000,3.337003e-01,1.000000,0.225615,502
1,0x06b51c6882b27cb05e712185531c1f74996dd988,0,4,4,0,0.024302,91256.000000,266363,1743,0.000000,0.000891,5.451070e-07,0.000167,0.793072,358
2,0x0795732aacc448030ef374374eaae57d2965c16c,0,3,3,0,0.018600,114674.666667,248424,1702,0.000000,0.000446,4.172106e-07,0.000139,0.774385,354
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,1879,28,24,4,270.312981,131456.107143,2433893,309,0.656534,0.011586,6.063304e-03,0.003470,0.139471,100
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,63,4,4,0,0.026005,91256.000000,266363,1673,0.022013,0.000891,5.833029e-07,0.000167,0.761167,348
